In [ ]:
!pip install prophet

Prophet is a tool used for time series forecasting, and we chose it among other models because not only is it very interpretable and user-friendly, but it can handle data which displays trends and seasonality, which is commonly observed in sales data. We first convert the purchase_date column to a datetime format as required. A custom function, forecast_sales, is defined to prepare the data by grouping sales by purchase_date and summing quantity_purchased. The data is then formatted for Prophet, with ds as the date column and y as the target variable to match Prophet's requirements. The Prophet model is fitted with the historical data and predicts sales for the next 30 days. The total forecasted demand is calculated by summing the predicted values (yhat). The dataset is grouped by item_name, store_region and supplier, applying the forecasting function to each group, resulting in a new dataframe containing forecasted demand for each item_name-store_region-supplier trio. This helps optimize inventory planning based on predicted future demand.

Economic order quantity (EOQ), also known as financial purchase quantity or economic buying quantity, is the order quantity that minimizes the total holding costs and ordering costs in inventory management. The following code calculates the EOQ for each (item_name, store_region, supplier) group based on forecasted demand. calculate_eoq is a function defined to extract the forecasted demand (D), corresponding unit price (S) and inventory holding cost (H) for each (item_name, store_region, supplier) combination, which is required in the standard EOQ formula sqrt[(2xDxS)/H]. The calculated EOQ values are applied to each row of grouped_forecasts and column 'EOQ' is created to store the results, allowing for the particular store or business to optimise inventory based on predicted future demand in the next step.

We capitalise on such model (EOQ formula) to incorporate the forecasted demand and current inventory levels in order to calculate reorder quantity if forecasted demand exceeds current inventory within a time frame of 30 days. We start by leftjoin merging on (item_name, store_region and supplier) the grouped_forecasts DataFrame, which contains forecasted demand and EOQ, with the relevant inventory information (inventory_level) from the original df DataFrame. A new column 'reorder_quantity' is then calculated such that if the forecasted demand exceeds the current inventory_level, the EOQ value is assigned as the reorder quantity; otherwise, it remains 0. The result is previewed using print(grouped_data.head()), providing insight into whether additional inventory is needed based on forecasted demand. For example, as seen from the print(grouped_data.head()) below, the store or business in BARISAL is encouraged to stock up ~166 quantities of 100% Juice Box variety 6.75 oz from supplier BIGSO AB so as to optimise inventory for the next 30 days


In [ ]:
import pandas as pd
from prophet import Prophet

df = pd.read_csv('final.csv', delimiter=",", encoding='ISO-8859-1')

# ensuring Purchase_date is of the right type
df['purchase_date'] = pd.to_datetime(df['purchase_date'])

# note: we chose prophet model among others because not only is it very interpretable and user-friendly,
#       but it can handle data which displays trends and seasonality, which is commonly observed in sales data.

# function to apply Prophet model for each group
def forecast_sales(group):
    # Prepare the data for Prophet
    sales_data = group.groupby('purchase_date')['quantity_purchased'].sum().reset_index()
    sales_data.columns = ['ds', 'y']  # must be 'ds' for date and 'y' for target to meet prophet requirements

    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(sales_data)

    # predict next 30 days/1 month
    forecast = model.predict(model.make_future_dataframe(periods=30))

    # Return the forecast and the sum of future sales (demand forecast for EOQ)
    forecasted_demand = forecast['yhat'].sum()  # Total forecasted demand
    return forecasted_demand

# Group by 'item_name', 'store_region', 'supplier' combi, then apply the forecast_sales function
grouped_forecasts = df.groupby(['item_name', 'store_region', 'supplier']).apply(forecast_sales).reset_index()
grouped_forecasts.columns = ['item_name', 'store_region', 'supplier', 'forecasted_demand']

# preview
print(grouped_forecasts.head())


In [ ]:
grouped_forecasts

,item_name,store_region,supplier,forecasted_demand
0,100% Juice Box Variety 6.75 oz,BARISAL,BIGSO AB,1845.635737
1,100% Juice Box Variety 6.75 oz,CHITTAGONG,BIGSO AB,4777.988366
2,100% Juice Box Variety 6.75 oz,DHAKA,BIGSO AB,10164.675577
3,100% Juice Box Variety 6.75 oz,KHULNA,BIGSO AB,2892.589393
4,100% Juice Box Variety 6.75 oz,RAJSHAHI,BIGSO AB,2921.920291
...,...,...,...,...
1843,Zoo Animal Cookies/Crackers,DHAKA,Indo Count Industries Ltd,9160.791257
1844,Zoo Animal Cookies/Crackers,KHULNA,Indo Count Industries Ltd,2890.647677
1845,Zoo Animal Cookies/Crackers,RAJSHAHI,Indo Count Industries Ltd,2992.582356
1846,Zoo Animal Cookies/Crackers,RANGPUR,Indo Count Industries Ltd,2085.196450


In [ ]:
import numpy as np
# Economic order quantity (EOQ) is a formula that helps businesses determine the ideal order size for inventory to minimize costs and meet demand
# Define a function to calculate EOQ based on forecasted demand
def calculate_eoq(row):
    # extract the forecasted demand
    D = row['forecasted_demand']

    # corresponding unit price and inventory cost for the item_name, store_region and supplier
    S = df.loc[(df['item_name'] == row['item_name']) & (df['supplier'] == row['supplier']) & (df['store_region'] == row['store_region']), 'unit_price'].values[0]
    H = df.loc[(df['item_name'] == row['item_name']) & (df['supplier'] == row['supplier']) & (df['store_region'] == row['store_region']), 'inventory_cost'].values[0]

    # calculate EOQ using the standard EOQ formula
    eoq = np.sqrt((2*S*D)/H)
    return eoq

# Apply the EOQ calculation to each product-store_region-supplier group
grouped_forecasts['EOQ'] = grouped_forecasts.apply(calculate_eoq, axis=1)

# forecasted demand and EOQ preview
print(grouped_forecasts.head())


                         item_name store_region  supplier  forecasted_demand  \
0  100% Juice Box Variety 6.75 oz       BARISAL  BIGSO AB        1845.635737   
1  100% Juice Box Variety 6.75 oz    CHITTAGONG  BIGSO AB        4777.988366   
2  100% Juice Box Variety 6.75 oz         DHAKA  BIGSO AB       10164.675577   
3  100% Juice Box Variety 6.75 oz        KHULNA  BIGSO AB        2892.589393   
4  100% Juice Box Variety 6.75 oz      RAJSHAHI  BIGSO AB        2921.920291   

          EOQ  
0  166.386706  
1  267.712206  
2  390.474242  
3  208.299882  
4  209.353300  


In [ ]:
grouped_forecasts

,item_name,store_region,supplier,forecasted_demand,EOQ
0,100% Juice Box Variety 6.75 oz,BARISAL,BIGSO AB,1845.635737,166.386706
1,100% Juice Box Variety 6.75 oz,CHITTAGONG,BIGSO AB,4777.988366,267.712206
2,100% Juice Box Variety 6.75 oz,DHAKA,BIGSO AB,10164.675577,390.474242
3,100% Juice Box Variety 6.75 oz,KHULNA,BIGSO AB,2892.589393,208.299882
4,100% Juice Box Variety 6.75 oz,RAJSHAHI,BIGSO AB,2921.920291,209.353300
...,...,...,...,...,...
1843,Zoo Animal Cookies/Crackers,DHAKA,Indo Count Industries Ltd,9160.791257,168.820139
1844,Zoo Animal Cookies/Crackers,KHULNA,Indo Count Industries Ltd,2890.647677,94.832094
1845,Zoo Animal Cookies/Crackers,RAJSHAHI,Indo Count Industries Ltd,2992.582356,96.489669
1846,Zoo Animal Cookies/Crackers,RANGPUR,Indo Count Industries Ltd,2085.196450,80.543639


In [ ]:
# Join the current inventory level with the EOQ data
grouped_data = pd.merge(grouped_forecasts, df[['item_name', 'store_region', 'supplier', 'inventory_level']].drop_duplicates(),
                        on=['item_name', 'store_region', 'supplier'], how='left')

# Calculate reorder quantity if forecasted demand exceeds current inventory
grouped_data['reorder_quantity'] = grouped_data.apply(
    lambda row: row['EOQ'] if row['forecasted_demand'] > row['inventory_level'] else 0, axis=1)

grouped_data = grouped_data.drop(columns=['inventory_level'])

results = grouped_data.groupby(['item_name', 'store_region', 'supplier', 'forecasted_demand', 'EOQ', 'reorder_quantity'], as_index=False).first()

# Preview
print(results.head())


                         item_name store_region  supplier  forecasted_demand  \
0  100% Juice Box Variety 6.75 oz       BARISAL  BIGSO AB        1845.635737   
1  100% Juice Box Variety 6.75 oz    CHITTAGONG  BIGSO AB        4777.988366   
2  100% Juice Box Variety 6.75 oz         DHAKA  BIGSO AB       10164.675577   
3  100% Juice Box Variety 6.75 oz        KHULNA  BIGSO AB        2892.589393   
4  100% Juice Box Variety 6.75 oz      RAJSHAHI  BIGSO AB        2921.920291   

          EOQ  reorder_quantity  
0  166.386706        166.386706  
1  267.712206        267.712206  
2  390.474242        390.474242  
3  208.299882        208.299882  
4  209.353300        209.353300  


In [ ]:
results

,item_name,store_region,supplier,forecasted_demand,EOQ,reorder_quantity
0,100% Juice Box Variety 6.75 oz,BARISAL,BIGSO AB,1845.635737,166.386706,166.386706
1,100% Juice Box Variety 6.75 oz,CHITTAGONG,BIGSO AB,4777.988366,267.712206,267.712206
2,100% Juice Box Variety 6.75 oz,DHAKA,BIGSO AB,10164.675577,390.474242,390.474242
3,100% Juice Box Variety 6.75 oz,KHULNA,BIGSO AB,2892.589393,208.299882,208.299882
4,100% Juice Box Variety 6.75 oz,RAJSHAHI,BIGSO AB,2921.920291,209.353300,209.353300
...,...,...,...,...,...,...
1843,Zoo Animal Cookies/Crackers,DHAKA,Indo Count Industries Ltd,9160.791257,168.820139,168.820139
1844,Zoo Animal Cookies/Crackers,KHULNA,Indo Count Industries Ltd,2890.647677,94.832094,94.832094
1845,Zoo Animal Cookies/Crackers,RAJSHAHI,Indo Count Industries Ltd,2992.582356,96.489669,96.489669
1846,Zoo Animal Cookies/Crackers,RANGPUR,Indo Count Industries Ltd,2085.196450,80.543639,80.543639


In [ ]:
#same question as above but incorporating the use of an API to output reorder_quantity from (item_name, store_region, supplier) input

#!pip install flask

import pandas as pd
import numpy as np
from prophet import Prophet
from flask import Flask, request, jsonify
import threading

# Initialize Flask app
app = Flask(__name__)


# Convert the data to a dictionary for quick lookup
reorder_data = grouped_data.set_index(['item_name', 'store_region', 'supplier'])['reorder_quantity'].to_dict()




In [ ]:

# Define the API endpoint
@app.route('/get_reorder_quantity', methods=['POST'])
def get_reorder_quantity():
    data = request.get_json()
    item_name = data.get('item_name')
    store_region = data.get('store_region')
    supplier = data.get('supplier')

    # Look up reorder quantity
    key = (item_name, store_region, supplier)
    reorder_quantity = reorder_data.get(key, "Unable to find item or store or supplier!")

    # Return the result as JSON
    return jsonify({"item_name": item_name, "store_region": store_region, "supplier": supplier, "reorder_quantity": reorder_quantity})





In [ ]:
# Run Flask in a separate thread
def run_app():
    app.run(host='0.0.0.0', port=5005)

thread = threading.Thread(target=run_app)
thread.start() # threading allows for the script to continue without blocking

!pip install pyngrok # Ngrok is used to create a public URL for this local server. This allows external access to the locally hosted API.

from pyngrok import ngrok

# Set your authentication token
ngrok.set_auth_token("2oYfROCsDsEwP5Cri2FtwY7pra8_B6ju2CrXwEac8sQVMSWa")

# Open a tunnel to the local server
public_url = ngrok.connect(5005)
print("Public URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://172.28.0.12:5005
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://86a6-35-231-180-234.ngrok-free.app" -> "http://localhost:5005"


In [ ]:

# USAGE

import requests

# for businesses to key in their desired input
input_data = {
    "item_name": "Zoo Animal Cookies/Crackers",
    "store_region": "KHULNA",
    "supplier": "Indo Count Industries Ltd"
}

# copy and paste the first link (ngrok) in the above public URL output in requests.post infront of /get_reorder_quantity
response = requests.post(f"https://86a6-35-231-180-234.ngrok-free.app//get_reorder_quantity", json=input_data)

# Print the response
print(response.json())

INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 18:02:29] "POST /get_reorder_quantity HTTP/1.1" 200 -


{'item_name': 'Zoo Animal Cookies/Crackers', 'reorder_quantity': 94.83209427566426, 'store_region': 'KHULNA', 'supplier': 'Indo Count Industries Ltd'}
